In [44]:
import os
import math
import time
import concurrent.futures
import multiprocessing
from tqdm import tqdm  # Importing tqdm for progress bar
from pydub import AudioSegment  # Import pydub for handling MP3, WAV, FLAC
SUPPORTED_AUDIO_FORMATS = ['.mp3', '.wav', '.flac']


In [45]:
def process_audio_file(audio_file, input_directory, output_directory):
    # Your logic to process each audio file
    input_file_path = os.path.join(input_directory, audio_file)
    audio = AudioSegment.from_file(input_file_path)
    
    # Example: Chopping the audio into 10-minute chunks
    chunk_length_ms = 10 * 60 * 1000  # 10 minutes in milliseconds
    audio_length = len(audio)
    chunks = [audio[i:i+chunk_length_ms] for i in range(0, audio_length, chunk_length_ms)]
    
    # Saving chunks
    for i, chunk in enumerate(chunks):
        output_file_path = os.path.join(output_directory, f"{audio_file}_chunk_{i}.wav")
        chunk.export(output_file_path, format="wav")


In [46]:
# 1. Sequential Approach
def chop_audio_sequential(input_directory, output_directory):
    start_time = time.time()
    audio_files = [f for f in os.listdir(input_directory) if any(f.endswith(ext) for ext in SUPPORTED_AUDIO_FORMATS)]

    for audio_file in tqdm(audio_files, desc="Processing Sequential", unit="file"):
        process_audio_file(audio_file, input_directory, output_directory)

    end_time = time.time()
    total_time = end_time - start_time
    return total_time, len(audio_files)


In [47]:
# 2. Multithreaded Approach
def chop_audio_threading(input_directory, output_directory):
    start_time = time.time()
    audio_files = [f for f in os.listdir(input_directory) if any(f.endswith(ext) for ext in SUPPORTED_AUDIO_FORMATS)]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for _ in tqdm(executor.map(lambda audio_file: process_audio_file(audio_file, input_directory, output_directory), audio_files), 
                      desc="Processing Threading", unit="file", total=len(audio_files)):
            pass

    end_time = time.time()
    total_time = end_time - start_time
    return total_time, len(audio_files)

In [48]:
def chop_audio_multiprocessing(input_directory, output_directory):
    audio_files = [f for f in os.listdir(input_directory) if any(f.endswith(ext) for ext in SUPPORTED_AUDIO_FORMATS)]
    
    start_time = time.time()

    with multiprocessing.Pool() as pool:
        # Use map instead of imap for easier handling with tqdm
        list(tqdm(pool.starmap(process_audio_file, [(audio_file, input_directory, output_directory) for audio_file in audio_files]),
                  desc="Processing Multiprocessing", unit="file", total=len(audio_files)))
    
    end_time = time.time()
    return end_time - start_time, None



In [2]:
import os

input_directory = "/home/mahmoud/Desktop/Projects/parallelproject/pre processing /multiprocessing_output"
output_directory_sequential = "/home/mahmoud/Desktop/Projects/parallelproject/segmenteddataset/segmentedseq_sequential"
output_directory_threading = "/home/mahmoud/Desktop/Projects/parallelproject/segmenteddataset/segmentedseq_threading"
output_directory_multiprocessing = "/home/mahmoud/Desktop/Projects/parallelproject/segmenteddataset/segmentedseq_multiprocessing"

# Create output directories if they do not exist
os.makedirs(output_directory_sequential, exist_ok=True)
os.makedirs(output_directory_threading, exist_ok=True)
os.makedirs(output_directory_multiprocessing, exist_ok=True)

# Get the number of CPU cores dynamically
num_processors = os.cpu_count()

# Sequential Approach
sequential_time, total_files = chop_audio_sequential(input_directory, output_directory_sequential)

# Threading Approach
parallel_time_threading, _ = chop_audio_threading(input_directory, output_directory_threading)

# Multiprocessing Approach
parallel_time_multiprocessing, _ = chop_audio_multiprocessing(input_directory, output_directory_multiprocessing)

# Calculate Metrics
calculate_metrics(sequential_time, parallel_time_threading, parallel_time_multiprocessing, total_files, num_processors=num_processors)


NameError: name 'chop_audio_sequential' is not defined